In [5]:
pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\015948036\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv


In [22]:
import time
# Step 1: Scrape the main site to get subcategory links dynamically
main_url = "https://www.calories.info/"
response = requests.get(main_url)
soup = BeautifulSoup(response.content, "html.parser")

# Step 2: Extract all <a> tags that link to subcategory pages under "/food/"
subcategory_links = []
for a_tag in soup.find_all("a", href=True):
    href = a_tag["href"]
    if href.startswith("/food/") and href.count("/") == 2:  # to avoid deeper links
        subcategory_links.append({
            "Subcategory": a_tag.get_text(strip=True),
            "URL": "https://www.calories.info" + href
        })

# Convert to DataFrame for easier handling
df_subcategories = pd.DataFrame(subcategory_links).drop_duplicates(subset=["URL"])

# Step 3: Visit each subcategory URL and scrape the table
scraped_data = []

for idx, row in df_subcategories.iterrows():
    subcategory_name = row["Subcategory"]
    subcategory_url = row["URL"]

    try:
        resp = requests.get(subcategory_url)
        soup = BeautifulSoup(resp.content, "html.parser")
        table = soup.find("table")
        if table:
            rows = table.find("tbody").find_all("tr")
            for r in rows:
                cols = r.find_all("td")
                if len(cols) >= 3:
                    food = cols[0].get_text(strip=True)
                    serving = cols[1].get_text(strip=True)
                    calories = cols[2].get_text(strip=True)
                    scraped_data.append({
                        "Subcategory": subcategory_name,
                        "Food": food,
                        "Serving": serving,
                        "Calories": calories,
                        "URL": subcategory_url
                    })
        time.sleep(0.1)
    except Exception as e:
        print(f"Failed to scrape {subcategory_url}: {e}")

# Step 4: Save to DataFrame and CSV
df_scraped = pd.DataFrame(scraped_data)
output_final_path = "all_calories_from_dynamic_links.csv"
df_scraped.to_csv(output_final_path, index=False)